# TL;DR

This notebook is used to do a unit test on a single server machine (kvmp-3).

Network topology setup as defined in `config/cluster/unittest-1.json`:
- 1 client VM
- 1 edge router VM
- 1 load balancer VM
- 4 application server VMs with different capacities
- Wikipedia trace hour 0

# Import libs

In [1]:
import sys
import os
utils_dir = '../../src/utils'
sys.path.insert(0, utils_dir) # add utils dir to path
import testbed_utils as tu
import time

# Run

In [30]:
# initialize params
config = 'unittest'
method = 'aquarius_ecmp_time'
trace = 'wiki'
sample = 'hour0.csv'
ep = 0
experiment = 'unittest'
clip_n = 20000
task_name, task_dir, nodes = tu.init_task_info(
    experiment=experiment,
    lb_method=method,
    trace=trace,
    sample=sample,
    cluster_config='{}-1.json'.format(config),
    alias=config,
)

In [31]:
# run all VMs on kvmp-3 locally
tu.prepare_img(lb_method=method, from_orig=True, debug_node=False)
tu.runall()

Create LB node image...
node_client_0 ready: ssh -p 8800 cisco@localhost
node_edge_0 ready: ssh -p 8850 cisco@localhost
node_lb_0 ready: ssh -p 8900 cisco@localhost
node_server_0 ready: ssh -p 9000 cisco@localhost
node_server_1 ready: ssh -p 9001 cisco@localhost
node_server_2 ready: ssh -p 9002 cisco@localhost
node_server_3 ready: ssh -p 9003 cisco@localhost


In [32]:
# check management channel between LBs and servers
net_ok = False
while not net_ok:
    try:
        tu.gt_socket_check()
        net_ok = True
    except:
        print('error')
        time.sleep(1)

LB Node 0: found error socket with server [3]
LB Node 0: found error socket with server [3]
LB Node 0: pass


In [33]:
# start processor agents on LBs
for lb in tu.NODES['lb']:
    lb.run_init_bg()

In [34]:
# prepare network trace
tu.prepare_trace_sample(trace, sample, clip_n=clip_n)

>> prepare trace sample: /home/yzy/aquarius/data/trace/wiki/hour0.csv


In [35]:
# run traffic
t0 = time.time()
tu.NODES['clt'][0].startTraffic()
print("total time: {:.3f}s".format(time.time()-t0))

total time: 46.591s


In [36]:
# fetch log results from LBs and application servers
for lb in tu.NODES['lb']:
    lb.fetch_result(task_dir, ep)

In [37]:
tu.NODES['clt'][0].fetch_result(task_dir, ep)

In [38]:
# shutdown VMs on kvmp-3 locally
tu.shutall()